In [1]:
import numpy as np
from matplotlib import pyplot as plt
import gsw  
import coare35vn

# Script to evaluate fluxes and length scales from bulk parameters
Using COARE 3.5 for most parameters and GSW for thermal expansion and haline contraction coefficients. Note that the code containing the main output function for COARE has been adjusted to only return the values we need here.

2023-04-26 _kaihc@met.no_ 

In [2]:
# Input data for COARE algorithm:
windspeed = 5.4
airTemperature = 7.8
relativeHumidity = 50.0
bulkSST = 7.4
latitude = 59.9
MSLP = 1011.1
Hs = 0.2
rain = 0.0 # mm/hour
mastHeight = 8.0
cloudCover = 7.0 # oktas
QswIdeal = 970.0 # https://www.clearskycalculator.com/ (Get shortwave estimate for correct time of day and position.)

# Extra input data for GSW
surfaceSalinity = 23.0
dBar = 0.1 # ie. depth 

# Reference density and gravity
g = 9.81
rho0 = 1025.0

In [3]:
# Calculate fluxes using COARE
Cfrac = cloudCover/8.0
albedo = 0.06
Qshortwave = (1.0-albedo)*QswIdeal*(1-Cfrac**3)
A = coare35vn.coare35vn(windspeed, airTemperature, relativeHumidity, bulkSST, P=MSLP, Rs=Qshortwave, sigH = Hs, zu=mastHeight, zt=mastHeight, zq=mastHeight, lat=latitude)
tau, Qsensible, Qlatent, LWnet, QnetMinusSW, evaporation = A[0].T


In [4]:
# Calculate thermal expansion and haline contraction coefficients
alpha = gsw.alpha(surfaceSalinity, bulkSST, dBar)
beta = gsw.beta(surfaceSalinity, bulkSST, dBar)
Cp = gsw.cp_t_exact(surfaceSalinity, bulkSST, dBar)

In [5]:
# Calculate the density flux
evap = evaporation/3600. # conversion from mm/hour to kg/s/m².
precip = rain/3600. # conversion from mm/hour to kg/s/m².
Qnet = -QnetMinusSW + Qshortwave
Ft = -Qnet/(rho0*Cp)
Fs = (evap-precip)/(1-surfaceSalinity/1000)
Qp = rho0*(alpha*Ft + beta*Fs) # density flux


In [6]:
# Calculate buoyancy flux and Obukhov length
Bf = -g*Qp/rho0
ustar = np.sqrt(tau/rho0) # friction velocity
kappa = 0.4 # von Karman constant
L = -ustar**3/(kappa*Bf)

In [7]:
# Print results.
print("Summary of fluxes defined as positive into the ocean:")
print("Sensible heat flux, %5.2f W/m²" % (-Qsensible))
print("Latent heat flux, %5.2f W/m²" % (-Qlatent))
print("Net longwave radiation, %5.2f W/m²" % (-LWnet))
print("Net heat flux, %5.2f W/m²" % (Qnet))
print("Buoyancy flux, %10.9f" % (Bf))
print("The Obhukov length is, %5.2f m" % (L))


Summary of fluxes defined as positive into the ocean:
Sensible heat flux,  4.07 W/m²
Latent heat flux, -57.00 W/m²
Net longwave radiation, 18.35 W/m²
Net heat flux, 266.38 W/m²
Buoyancy flux, -0.000000109
The Obhukov length is,  4.69 m
